In [1]:
import pandas as pd, skill_cat, multiprocessing.dummy
from jobs_skills_weights import *
import synonyms

In [2]:
skill_cat._refresh()
synonyms._refresh()

# What is the goal here?
For each job, create a dict from (a skill name that job wants) to (a float indicating the importance of that skill)

In [3]:
# What fraction of skill weight comes from being listed in a job description?
# 1-list_weight gives the weight of being in a category
list_weight = 4/5.0 

In [4]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)

In [5]:
skill_cat._refresh()
synonyms._refresh()

In [6]:
job_skill_text_data = pd.concat(
    {key: extract_skills_data(value) for key, value in raw_job_details.items()},
    names=["id"]
).groupby(level=[0,2,3]).sum().query("`skill`.isin(@skill_cat.skill_to_categories.keys())").sort_index()

jst_counts = job_skill_text_data.pop("count")
job_skill_text_data["share of job"] = jst_counts / jst_counts.groupby(level=0).sum()

In [7]:
jobs_categories_skills_data = pd.DataFrame([{
        "id":job_id,
        "category":category,
        "skill":skill_name,
        "share of job":len(categories_skill_is_in), # This skill will appear in `n` categories, so we'll divide it by `n` to keep it conserved.
    } for job_id in raw_job_details.keys()
        for skill_name, categories_skill_is_in in skill_cat.skill_to_categories.items()
            for category in categories_skill_is_in
]).set_index(["id", "category", "skill"])

In [8]:
jcst_data = (job_skill_text_data / jobs_categories_skills_data).reset_index().fillna({"share of job":0}) # jcst_data => jobs_categories_skills_text_data
jcst_data["skill text"].fillna(jcst_data["skill"].apply(skill_cat.skill_to_skill_title.get), inplace=True)
jcst_data = jcst_data.set_index(["id","category","skill","skill text"]).sort_index()

In [9]:
from skill_weights import category_biases

jcst_data["share of job"] = (jcst_data["share of job"] * (1-category_biases.sum())) + (category_biases/jcst_data.groupby(level=["id","category"]).size())

In [10]:
jcst_data

share of job
id                                   category skill            skill text                    
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science Computer Science           0.0
                                              experimentation  Experimentation            0.0
                                              research         Research                   0.0
                                     Admin    automation       Automation                 0.0
                                              aws              AWS                        0.0
...                                                                                       ...
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix             Unix                       0.0
                                     Visual   graphic design   Graphic Design             0.0
                                              usability        Usability                  0.0
                                     Webdev   javascript       Javascript                 0.0
                                              web development  Web Development            0.0

[12700 rows x 1 columns]

In [11]:
jcst_wc = (jcst_data*list_weight + (jcst_data*(1-list_weight)).groupby(level=[0,1]).mean())

In [12]:
jcst_wc

share of job
id                                   category skill            skill text                    
0124e218-2d33-41e8-907b-5228ea386455 Academic computer science Computer Science      0.000000
                                              experimentation  Experimentation       0.000000
                                              research         Research              0.000000
                                     Admin    automation       Automation            0.001569
                                              aws              AWS                   0.001569
...                                                                                       ...
fdd15faa-0804-4b07-9f55-715436213e75 Unix     unix             Unix                  0.000000
                                     Visual   graphic design   Graphic Design        0.000000
                                              usability        Usability             0.000000
                                     Webdev   javascript       Javascript            0.000000
                                              web development  Web Development       0.000000

[12700 rows x 1 columns]

In [13]:
jst_wc = jcst_wc.groupby(level=[0,2,3]).sum()

In [14]:
jst_wc.head(50)

share of job
id                                   skill                skill text                        
0124e218-2d33-41e8-907b-5228ea386455 agile                Agile                     0.007143
                                     algebra              Algebra                   0.002424
                                     algorithms           Algorithms                0.003450
                                     analysis             Analysis                  0.002667
                                     apache tinkerpop     Apache Tinkerpop          0.001667
                                     api design           API Design                0.001026
                                     automation           Automation                0.001569
                                     aws                  AWS                       0.004902
                                     aws neptune          AWS Neptune               0.001667
                                     azure databricks     Azure Databricks          0.003333
                                     bash                 Bash                      0.011333
                                     big data             Big Data                  0.006000
                                     bioinformatics       Bioinformatics            0.000000
                                     budget               Budget                    0.000000
                                     build tools          Build Tools               0.000000
                                     c                    C                         0.011333
                                     c++                  C++                       0.011333
                                     cancer               Cancer                    0.000000
                                     ci/cd                CI/CD                     0.001569
                                     cloud computing      Cloud Computing           0.001569
                                     coding               Coding                    0.012359
                                     collaboration        Collaboration             0.007143
                                     communication        Communication             0.007143
                                     computer engineering Computer Engineering      0.000000
                                     computer science     Computer Science          0.001026
                                     containerization     Containerization          0.001569
                                     curious              Curious                   0.007143
                                     data                 Data                      0.167667
                                     data driven          Data Driven               0.007143
                                     data engineering     Data Engineering          0.002667
                                     data manipulation    Data Manipulation         0.002667
                                     data pipelines       Data Pipelines            0.002667
                                     data science         Data Science              0.056000
                                     debugging            Debugging                 0.001026
                                     deep learning        Deep Learning             0.002424
                                     documentation        Documentation             0.001026
                                     end to end           End To End                0.054902
                                     experimentation      Experimentation           0.000000
                                     feature engineering  Feature Engineering       0.055758
                                     frameworks           Frameworks                0.002594
                                     genetics             Genetics                  0.000000
                                     git                  Git                    